# Evaluation Experiment Results

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pickle as pickle
import pandas as pd
import numpy as np

import copy

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset

from src.trainer import CaseDataSet
from src.model import DLModels
from src.trainer import Trainer
from src.utils import TorchUtils

In [2]:
torch_device, device_package = TorchUtils.get_torch_device()


## Load Data Set

In [55]:
data_folder_path = "../../Data/Training/"
embedding_type = ["w2v", "st", "onehot"]
data_set_type = ["source", "target"]
earliness_requirement = True
test_sets_list = {}
for data_type in data_set_type:
    test_sets_embedding_list = {}
    for embedding in embedding_type:   
        data_version = "_test_sorted"
        embedding_version = "_" + embedding                
        case_data_set = CaseDataSet.CaseDataset(project_data_path=data_folder_path,
                                                split_pattern="641620split",
                                                input_data=data_type,
                                                data_version=data_version,
                                                embedding_version=embedding_version,
                                                earliness_requirement=earliness_requirement)
        test_sets_embedding_list[embedding] = case_data_set
    test_sets_list[data_type] = test_sets_embedding_list
    

val_sets_list = {}
for data_type in data_set_type:
    val_sets_embedding_list = {}
    for embedding in embedding_type:   
        data_version = "_val_sorted"
        embedding_version = "_" + embedding                
        case_data_set = CaseDataSet.CaseDataset(project_data_path=data_folder_path,
                                                split_pattern="641620split",
                                                input_data=data_type,
                                                data_version=data_version,
                                                embedding_version=embedding_version,
                                                earliness_requirement=earliness_requirement)
        val_sets_embedding_list[embedding] = case_data_set
    val_sets_list[data_type] = val_sets_embedding_list

## Load Models

In [56]:
model_hyper_para = {"w2v": {"input_size": 51,
                            "hidden_size": 128,
                            "num_layers": 1},
                    "st": {"input_size": 385,
                           "hidden_size": 512,
                           "num_layers": 1},
                    "onehot": {"input_size": 48,
                               "hidden_size": 128,
                               "num_layers": 1}}
model_folder_path = "../../Model/641620split/LSTM/LSTM"
embedding_type = ["w2v", "st", "onehot"]
model_type = ["source", "target"]
model_list = {}
for training_set_type in model_type:
    model_embedding_list = {}
    for embedding in embedding_type:
        model_experiment_list = []
        for i in range(10):
            training_prefix = "_T"
            if training_set_type == "source":
                training_prefix = "_S"
            hyper_para_prefix = "_h" + str(model_hyper_para[embedding]["hidden_size"]) + "_l1"
            file_name = model_folder_path + training_prefix + hyper_para_prefix + "_" + embedding + "_" + str(i) + ".LSTM"
            model = torch.load(file_name)
            model_experiment_list.append(model)
        model_embedding_list[embedding] = model_experiment_list
    model_list[training_set_type] = model_embedding_list

In [61]:
scale_model_folder_path = "../../Model/scale/LSTM/LSTM"
scale_model_list = {}
scale_ratio = ["0.01" ,"0.05", "0.1", "0.5"]
for ratio in scale_ratio:
    model_embedding_list = {}
    for embedding in ["w2v"]:
        model_experiment_list = []
        for i in range(10):
            training_prefix = "_T"
            hyper_para_prefix = "_h" + str(model_hyper_para[embedding]["hidden_size"]) + "_l1"
            file_name = scale_model_folder_path + training_prefix + hyper_para_prefix + "_" + embedding + "_" + ratio + "_r" + str(i) + ".LSTM"
            model = torch.load(file_name)
            model_experiment_list.append(model)
        model_embedding_list[embedding] = model_experiment_list
    scale_model_list[ratio] = model_embedding_list

## Evaluate

### Transfer Effect

In [58]:
res_list = []
for embedding in embedding_type:
    print("Evaluating embedding: ", embedding)
    for training_type in model_type:
        print("Evaluating model type: ", training_type)
        for i in range(10):
            print("Evaluating model index: ", i)
            for test_type in data_set_type:
                model = model_list[training_type][embedding][i]
                test_set = val_sets_list[test_type][embedding]
                res = Trainer.eval_model(model, test_set, torch_device,
                                         device_package, decision_boundary=0.5,
                                         weighted=True, print_res=False)
                res_pd = pd.DataFrame(columns=["ROC_AUC", "Weighted F1 score",
                                               "Weighted precision", "Weighted recall"],
                                      data = np.array(res).reshape((1,-1)))
                res_pd["Embedding"] = embedding
                res_pd["Training set"] = training_type
                res_pd["Testing set"] = test_type
                res_pd["Model index"] = i
                res_list.append(res_pd)

transfer_res_pd = pd.concat(res_list).reset_index(drop=True)
res_folder_path = "../../Result/641620split/LSTM/"
transfer_res_pd.to_excel(res_folder_path+"Transfer_Val_weighted.xlsx")

Evaluating embedding:  w2v
Evaluating model type:  source
Evaluating model index:  0
Evaluating model index:  1
Evaluating model index:  2
Evaluating model index:  3
Evaluating model index:  4
Evaluating model index:  5
Evaluating model index:  6
Evaluating model index:  7
Evaluating model index:  8
Evaluating model index:  9
Evaluating model type:  target
Evaluating model index:  0
Evaluating model index:  1
Evaluating model index:  2
Evaluating model index:  3
Evaluating model index:  4
Evaluating model index:  5
Evaluating model index:  6
Evaluating model index:  7
Evaluating model index:  8
Evaluating model index:  9
Evaluating embedding:  st
Evaluating model type:  source
Evaluating model index:  0
Evaluating model index:  1
Evaluating model index:  2
Evaluating model index:  3
Evaluating model index:  4
Evaluating model index:  5
Evaluating model index:  6
Evaluating model index:  7
Evaluating model index:  8
Evaluating model index:  9
Evaluating model type:  target
Evaluating mo

In [6]:
res_folder_path = "../../Result/641620split/LSTM/"
transfer_res_pd.to_excel(res_folder_path+"Transfer_Test_weighted.xlsx")

### Scale Test

In [66]:
scale_res_list = []
for ratio in scale_ratio:
    print(ratio)
    for embedding in ["w2v"]:
        for i in range(10):
            model = scale_model_list[ratio][embedding][i]
            test_set = test_sets_list["target"][embedding]
            res = Trainer.eval_model(model, test_set, torch_device,
                                     device_package, decision_boundary=0.5,
                                     weighted=True, print_res=False)
            res_pd = pd.DataFrame(columns=["ROC_AUC", "Weighted F1 score",
                                           "Weighted precision", "Weighted recall"],
                                  data = np.array(res).reshape((1,-1)))
            res_pd["Embedding"] = embedding
            res_pd["Training set"] = "target_" + ratio
            res_pd["Testing set"] = "target"
            res_pd["Model index"] = i
            scale_res_list.append(res_pd)

scale_res_pd = pd.concat(scale_res_list).reset_index(drop=True)
res_folder_path = "../../Result/641620split/LSTM/"
scale_res_pd.to_excel(res_folder_path+"Scale_test_weighted_w2v.xlsx")

0.01
0.05
0.1
0.5


In [12]:
res_folder_path = "../../Result/641620split/LSTM/"
scale_res_pd.to_excel(res_folder_path+"Scale_Test_weighted.xlsx")

In [11]:
scale_res_pd

,ROC_AUC,Weighted F1 score,Weighted precision,Weighted recall,Embedding,Training set,Testing set,Model index
0,0.535455,0.562171,0.606350,0.539418,w2v,target_0.01,target,0
1,0.576242,0.487172,0.663985,0.474673,w2v,target_0.01,target,1
2,0.532818,0.425265,0.665440,0.430252,w2v,target_0.01,target,2
3,0.517964,0.312423,0.726307,0.370971,st,target_0.01,target,0
4,0.551911,0.395942,0.691737,0.416025,st,target_0.01,target,1
5,0.591895,0.370037,0.709060,0.402665,st,target_0.01,target,2
6,0.535027,0.623049,0.605711,0.653234,onehot,target_0.01,target,0
7,0.576190,0.518073,0.660549,0.499059,onehot,target_0.01,target,1
8,0.542652,0.530450,0.623179,0.505068,onehot,target_0.01,target,2
9,0.538314,0.624399,0.609158,0.648799,w2v,target_0.05,target,0


## Analysis

In [51]:
grouped_exper = transfer_res_pd.groupby(["Embedding", "Training set", "Testing set"])
group_exper_stat = grouped_exper[["ROC_AUC", "Weighted F1 score", "Weighted precision", "Weighted recall"]].apply(lambda x: np.mean(x, axis=0))

In [52]:
group_exper_stat[["Std AUC_ROC",
                 "Std Weighted F1 score",
                 "Std Weighted precision",
                 "Std Weighted recall"]] = grouped_exper[["ROC_AUC",
                                                        "Weighted F1 score",
                                                        "Weighted precision",
                                                        "Weighted recall"]].apply(lambda x: np.std(x, axis=0))

In [54]:
group_exper_stat.to_excel(res_folder_path+"Scale_Test_weighted_agg.xlsx")

In [49]:
grouped_exper[["ROC_AUC", "Weighted F1 score", "Weighted precision", "Weighted recall"]].apply(lambda x: x)

ROC_AUC  Weighted F1 score  \
Embedding Training set Testing set                                   
onehot    source       source      80  0.704534           0.740803   
                                   82  0.696500           0.750452   
                                   84  0.707936           0.738002   
                                   86  0.695451           0.719462   
                                   88  0.684784           0.728792   
...                                         ...                ...   
w2v       target       target      31  0.609029           0.410310   
                                   33  0.606099           0.569504   
                                   35  0.627638           0.598759   
                                   37  0.609128           0.593824   
                                   39  0.625923           0.519940   

                                       Weighted precision  Weighted recall  
Embedding Training set Testing set                                          
onehot    source       source      80            0.749500         0.734286  
                                   82            0.746177         0.756120  
                                   84            0.744227         0.732977  
                                   86            0.730993         0.711007  
                                   88            0.734241         0.724227  
...                                                   ...              ...  
w2v       target       target      31            0.762820         0.436689  
                                   33            0.646750         0.545222  
                                   35            0.661590         0.576223  
                                   37            0.649935         0.571379  
                                   39            0.692281         0.505809  

[120 rows x 4 columns]